In [ ]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mticks
get_ipython().run_line_magic('matplotlib', 'inline')
from matplotlib.ticker import ScalarFormatter

In [ ]:
# Reading in the raw datasets and storing in a dataframe.
confirmed = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
recovered = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv")
deaths = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")

# Cleaning the dataframes and handling dates correctly, also calculating totals for graphs
confirmed = confirmed.drop(columns=['Lat','Long'])
confirmed = confirmed.groupby('Country/Region').sum()
confirmed = confirmed.transpose()
confirmed = confirmed[confirmed >= 0].dropna()
confirmed['Date'] = pd.to_datetime(confirmed.index)
confirmed = confirmed.sort_values(by=['Date'])
confirmed = confirmed.set_index("Date")
all_confirmed = confirmed.sum(axis = 1)

recovered = recovered.drop(columns=['Lat','Long'])
recovered = recovered.groupby('Country/Region').sum()
recovered = recovered.transpose()
recovered = recovered[recovered >= 0].dropna()
recovered['Date'] = pd.to_datetime(recovered.index)
recovered = recovered.sort_values(by=['Date'])
recovered = recovered.set_index("Date")
all_recovered = recovered.sum(axis = 1)

deaths = deaths.drop(columns=['Lat','Long'])
deaths = deaths.groupby('Country/Region').sum()
deaths = deaths.transpose()
deaths = deaths[deaths >= 0].dropna()
deaths['Date'] = pd.to_datetime(deaths.index)
deaths = deaths.sort_values(by=['Date'])
deaths = deaths.set_index("Date")
all_deaths = deaths.sum(axis = 1)

# Preparing dataframes to be plotted, removing unnecessary data such as countries with less than 100 cases/10 deaths or recoveries
plt.rcParams['axes.prop_cycle'] = plt.cycler(color = ['blue', 'red', 'green','darkblue', 'orange', 'brown', 'magenta', 'purple'])
highlighted_confirmed = confirmed[['United Kingdom', 'US', 'India', 'Italy', 'China', 'Russia', 'Brazil', 'Spain']]
grey_confirmed = confirmed.drop(['United Kingdom', 'US', 'India', 'Italy', 'China', 'Russia', 'Brazil', 'Spain'], axis = 1)
grey_confirmed = grey_confirmed.drop(columns = grey_confirmed.columns[grey_confirmed.iloc[-1].lt(100)])

highlighted_recovered = recovered[['United Kingdom', 'US', 'India', 'Italy', 'China', 'Russia', 'Brazil', 'Spain']]
grey_recovered = recovered.drop(['United Kingdom', 'US', 'India', 'Italy', 'China', 'Russia', 'Brazil', 'Spain'], axis = 1)
grey_recovered = grey_recovered.drop(columns = grey_recovered.columns[grey_recovered.iloc[-1].lt(10)])

highlighted_deaths = deaths[['United Kingdom', 'US', 'India', 'Italy', 'China', 'Russia', 'Brazil', 'Spain']]
grey_deaths = deaths.drop(['United Kingdom', 'US', 'India', 'Italy', 'China', 'Russia', 'Brazil', 'Spain'], axis = 1)
grey_deaths = grey_deaths.drop(columns = grey_deaths.columns[grey_deaths.iloc[-1].lt(10)])

highlighted_test = recovered[['United Kingdom']]

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))

# Plotting the countries in grey
for country in grey_confirmed.columns:
    country1 = grey_confirmed[country]
    country1 = country1[country1 >= 100]
    while any(country1.diff() < 0):
        country1 = country1[country1.diff() > 0]
    ax.plot(country1.values, country1.diff(), color = 'lightgrey')
        
# Plotting the highlighted countries and removing any values that are negative or not needed
for country in highlighted_confirmed.columns:
    country1 = highlighted_confirmed[country]
    country1 = country1[country1 >= 100]
    while any(country1.diff() < 0):
        country1 = country1[country1.diff() > 0]
    ax.plot(country1.values, country1.diff(), marker = '.', markevery = 2, Label = country)
    
plt.arrow(300, 1000, 10000, 15000, color='black', head_width = 3500)
ax.annotate("Exponential Growth", xy = (750, 2500), rotation = 19, fontsize = 18)

# Setting up Axes
plt.xlabel("Total Cases (log) by Time",fontsize = 22)
ax.set_xscale('log')
ax.set_xticks([100, 1000, 10000, 100000, 100000, 1000000])
ax.get_xaxis().set_major_formatter(mticks.ScalarFormatter())
ax.set_xlim(xmin = 100)

plt.ylabel("New Daily Cases (log) by Time", fontsize = 22)
ax.set_yscale('log')
ax.set_yticks([1, 10, 100, 1000, 10000, 100000])
ax.get_yaxis().set_major_formatter(mticks.ScalarFormatter())

ax.grid(which='minor', alpha=0.2)
ax.grid(which='major', alpha=0.5)
plt.legend(loc = 2)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))

# Plotting the countries in grey
for country in grey_recovered.columns:
    country1 = grey_recovered[country]
    country1 = country1.dropna()
    country1 = country1[country1 >= 10]
    while any(country1.diff() < 0):
        country1 = country1[country1.diff() > 0]
    ax.plot(country1.values, country1.diff(), color = 'lightgrey')
        
# Plotting the highlighted countries and removing any values that are negative or not needed
for country in highlighted_recovered.columns:
    country1 = highlighted_recovered[country]
    country1 = country1.dropna()
    country1 = country1[country1.values >= 10]
    while any(country1.diff() < 0):
        country1 = country1[country1.diff() > 0]
    ax.plot(country1.values, country1.diff(), marker = '.', markevery = 2, Label = country)
    
plt.arrow(300, 1000, 10000, 15000, color='black', head_width = 3500)
ax.annotate("Exponential Growth", xy = (750, 2500), rotation = 20, fontsize = 18)

# Setting up Axes
plt.xlabel("Total Recoveries (log) by Time",fontsize=22)
ax.set_xscale('log')
ax.set_xticks([10, 100, 1000, 10000, 100000, 100000, 1000000])
ax.get_xaxis().set_major_formatter(mticks.ScalarFormatter())
ax.set_xlim(xmin = 10)

plt.ylabel("New Daily Recoveries (log) by Time", fontsize=22)
ax.set_yscale('log')
ax.set_yticks([1, 10, 100, 1000, 10000, 100000])
ax.get_yaxis().set_major_formatter(mticks.ScalarFormatter())

ax.grid(which='minor', alpha=0.2)
ax.grid(which='major', alpha=0.5)
plt.legend(loc = 2)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))

# Plotting the countries in grey
for country in grey_deaths.columns:
    country1 = grey_deaths[country]
    country1 = country1[country1 >= 10]
    while any(country1.diff() < 0):
        country1 = country1[country1.diff() > 0]
    ax.plot(country1.values, country1.diff(), color = 'lightgrey')
        
# Plotting the highlighted countries and removing any values that are negative or not needed
for country in highlighted_deaths.columns:
    country1 = highlighted_deaths[country]
    country1 = country1[country1 >= 10]    
    while any(country1.diff() < 0):
        country1 = country1[country1.diff() > 0]
    ax.plot(country1.values, country1.diff(), marker = '.', markevery = 2, Label = country)

plt.arrow(300, 1000, 10000, 15000, color='black', head_width = 3500)
ax.annotate("Exponential Growth", xy = (750, 2500), rotation = 20, fontsize = 18)

# Setting up Axes
plt.xlabel("Total Deaths (log) by Time",fontsize=22)
ax.set_xscale('log')
ax.set_xticks([10, 100, 1000, 10000, 100000, 100000, 1000000])
ax.get_xaxis().set_major_formatter(mticks.ScalarFormatter())
ax.set_xlim(xmin = 10)

plt.ylabel("New Daily Deaths (log) by Time", fontsize=22)
ax.set_yscale('log')
ax.set_yticks([1, 10, 100, 1000, 10000, 100000])
ax.get_yaxis().set_major_formatter(mticks.ScalarFormatter())

ax.grid(which='minor', alpha=0.2)
ax.grid(which='major', alpha=0.5)
plt.legend(loc = 2)
plt.show()